In [15]:
# The same as before, but the winning value computation is slightly changed.
# Since we really care about the "possible win rate", that is, the times
# we've won out of all the games below, we will be updating the UCB computation,
# the backpropagation and the score calculation.
# See: https://en.wikipedia.org/wiki/Monte_Carlo_tree_search#Principle_of_operation
# I also want to test whether the AI plays to a draw against itself. At least,
# this might be a step in the right direction regarding testing the algorithm.

In [16]:
from copy import deepcopy
import numpy as np

def add_score(prev_score, to_add):
    return (prev_score[0] + to_add[0], prev_score[1] + to_add[1])

def equal_arrays(ar1, ar2):
    if len(ar1) != len(ar2):
        return False
    for i in range(len(ar1)):
        if ar1[i] != ar2[i]:
            return False
    return True

class XOX_Game():
    def __init__(self, board = None, player=1):
        if board is None:
            self.board = [[0,0,0],[0,0,0],[0,0,0]]
        else:
            self.board = board
        self.player = player
    def is_terminal(self):
        # Check if all of the board is full or not
        not_found_empty = True
        for i in range(3):
            for j in range(3):
                if self.board[i][j] == 0:
                    not_found_empty = False
                    break
        if not_found_empty:
            return True
        # Check rows
        for i in range(3):
            if equal_arrays(self.board[i], [1,1,1]) or equal_arrays(self.board[i], [2,2,2]):
                return True
        # Check columns
        transposed_board = list(np.transpose(np.array(self.board)))
        
        for i in range(3):
            if equal_arrays(transposed_board[i], [1,1,1]) or equal_arrays(transposed_board[i], [2,2,2]):
                return True
        # Check diagonals
        if self.board[0][0] == self.board[1][1] and self.board[2][2] == self.board[1][1] and (self.board[1][1] in [1,2]):
            return True
        if self.board[0][2] == self.board[1][1] and self.board[2][0] == self.board[1][1] and (self.board[1][1] in [1,2]):
            return True
        return False
    
    
    
    def get_score(self):
        # Scores are in the format:
        # (x_wincount, o_wincount)
        x_wins = (1,0)
        o_wins = (0,1)
        draw = (0,0)
        
        # Check rows
        
        for i in range(3):
            if equal_arrays(self.board[i], [1,1,1]):
                return x_wins
            elif equal_arrays(self.board[i], [2,2,2]):
                return o_wins
        
        # Check columns
        transposed_board = list(np.transpose(np.array(self.board)))
        
        for i in range(3):
            if equal_arrays(transposed_board[i], [1,1,1]):
                return x_wins
            elif equal_arrays(transposed_board[i], [2,2,2]):
                return o_wins
            
        # Check diagonals
        if self.board[0][0] == self.board[1][1] and self.board[2][2] == self.board[1][1] and (self.board[1][1] in [1,2]):
            if self.board[1][1] == 1:
                return x_wins
            else:
                return o_wins
        if self.board[0][2] == self.board[1][1] and self.board[2][0] == self.board[1][1] and (self.board[1][1] in [1,2]):
            if self.board[1][1] == 1:
                return x_wins
            else:
                return o_wins
        return draw
    def all_actions(self):
        if self.is_terminal():
            return []
        actions = []
        for i in range(3):
            for j in range(3):
                if self.board[i][j] == 0:
                    actions.append((i, j))
        return actions
    def play_action(self, action):
        def toggler(num):
            if num == 1:
                return 2
            return 1
        board = deepcopy(self.board)
        board[action[0]][action[1]] = self.player
        return XOX_Game(board, toggler(self.player))
    def all_states(self):
        actions = self.all_actions()
        states = []
        for action in actions:
            states.append(self.play_action(action))
        return states
                
    def __repr__(self):
        resArr = [['-','-','-'],['-','-','-'],['-','-','-']]
        
        for i in range(3):
            for j in range(3):
                if self.board[i][j] == 1:
                    resArr[i][j] = 'X'
                elif self.board[i][j] == 2:
                    resArr[i][j] = 'O'
        for i in range(3):
            resArr[i] = str(resArr[i])
        return f"Player {self.player}:\n" + "\n".join(resArr) 
            
      

In [17]:
g = XOX_Game()
print(g)

Player 1:
['-', '-', '-']
['-', '-', '-']
['-', '-', '-']


In [18]:
g.all_states()[0].all_states()[3].all_states()[0].all_states()[1].all_states()[0].get_score()

(1, 0)

In [19]:
# Seems fine. Now, onto the MCTS functions. We will edit UCB1 for the new score calculation.

In [24]:
from math import sqrt
from math import log
from random import choice

coeff = sqrt(2)

class MCTSNode():
    def __init__(self, parent, state):
        self.parent = parent
        self.state = state
        self.visitCount = 0
        self.score = (0,0)
        self.children = []
    def visit(self):
        self.visitCount += 1
    def setChildren(self, children):
        self.children = children
    def increment_score(self, newscore):
        self.score = add_score(self.score, newscore)
    def __repr__(self):
        return f"V:{self.visitCount}, S:{self.score}, \n" + str(self.state)
        
def backpropagate(leaf, score):
    node = leaf
    while not node is None:
        node.visit()
        node.increment_score(score) 
        node = node.parent
    # We will only use the scores in the ucb calculation and this will be sufficient.


def uct(node):
    # Taken from: https://en.wikipedia.org/wiki/Monte_Carlo_tree_search
    # w_i / n_i + c * sqrt(ln(N_i)/n_i)
    if node.visitCount == 0:
        return float('inf')
    if node.state.player == 1:
        wincount = node.score[0]
    else:
        wincount = node.score[1]
    if (node.parent is None) or (node.parent.visitCount == 0):
        return float('-inf')
    else:
        return (wincount / node.visitCount) + coeff * sqrt(log(node.parent.visitCount) / node.visitCount)


def selection(node):
    children = node.children
    #print(children)
    bestScore = 0
    bestChild = None
    for child in children:
        newScore = uct(child)
        if newScore > bestScore:
            bestScore = newScore
            bestChild = child
    return bestChild

def randomPlay(node):
    tempState = deepcopy(node.state)
    while not tempState.is_terminal():
        try:
            tempState = choice(tempState.all_states())
        except IndexError:
            print("What?")
            print(tempState)
    return tempState.get_score()

def expand(node):
    newStates = node.state.all_states()
    children = [MCTSNode(parent = node, state = s) for s in newStates]
    node.setChildren(children)
    
def descend(node):
    tempNode = node
    while len(tempNode.children) > 0:
        tempNode = selection(tempNode)
    return tempNode



In [25]:
# Fingers crossed..

In [26]:
def MCTS(gamestate, limit = 10000):
    # Iteration limit of 10k
    node = MCTSNode(parent = None, state = gamestate)
    iteration_count = 0
    while iteration_count < limit:
        iteration_count += 1
        node_to_expand = descend(node)
        if not node_to_expand.state.is_terminal():
            expand(node_to_expand)
            to_explore = choice(node_to_expand.children)
            result = randomPlay(to_explore)
            backpropagate(to_explore, result)
        else:
            backpropagate(node, node.state.get_score())
    return selection(node)
            

In [27]:
MCTS(gamestate = XOX_Game())

V:255, S:(117, 102), 
Player 2:
['-', '-', '-']
['X', '-', '-']
['-', '-', '-']

In [30]:
def comp_vs_comp():
    state = XOX_Game()
    while not state.is_terminal():
        print(state)
        node = MCTS(gamestate = state)
        state = node.state
    print(state)
    return


In [31]:
comp_vs_comp()

Player 1:
['-', '-', '-']
['-', '-', '-']
['-', '-', '-']
Player 2:
['-', 'X', '-']
['-', '-', '-']
['-', '-', '-']
Player 1:
['-', 'X', '-']
['-', '-', 'O']
['-', '-', '-']
Player 2:
['-', 'X', 'X']
['-', '-', 'O']
['-', '-', '-']
Player 1:
['-', 'X', 'X']
['-', '-', 'O']
['O', '-', '-']
Player 2:
['X', 'X', 'X']
['-', '-', 'O']
['O', '-', '-']
